RAG 기반의 질문 답변 기능을 파이썬 SDK 나 REST API 둘 중 하나로 구현해주세요.

## 1. Python SDK

In [155]:
import os
import base64
from openai import AzureOpenAI

def request_SDK(prompt):
    try:
        endpoint = os.getenv("ENDPOINT_URL", "https://7ai015-openai.openai.azure.com/")
        deployment = os.getenv("DEPLOYMENT_NAME", "7ai015-gpt-4o-mini")
        search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://7ai015-aisearch.search.windows.net")
        search_key = os.getenv("SEARCH_KEY", "QkjHtmZVcwnXNZDgY8uH46TkT5N3OkXauMa1OH0le2AzSeDXapGA")
        search_index = os.getenv("SEARCH_INDEX_NAME", "margies-index")
        subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "937jKABfHgomOxXzVz4q7cxqEFwjMqrZ1HxIr4nUs4SfYQqD2SQrJQQJ99BFACfhMk5XJ3w3AAABACOG8ETF")
        search_semantic = "margies-index-semantic-configuration"

        # 키 기반 인증을 사용하여 Azure OpenAI 클라이언트 초기화
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=subscription_key,
            api_version="2025-01-01-preview",
        )

        # IMAGE_PATH = "YOUR_IMAGE_PATH"
        # encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

        #채팅 프롬프트 준비
        chat_prompt = [
            {
                "role": "system",
                "content": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]

        # 음성이 사용되는 경우 음성 결과 포함
        messages = chat_prompt

        # 완료 생성
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.5,
            top_p=0.95,
            extra_body={
            "data_sources": [{
                "type": "azure_search",
                "parameters": {
                    "endpoint": f"{search_endpoint}",
                    "index_name": f"{search_index}",
                    "semantic_configuration": search_semantic,
                    "query_type": "semantic",
                    "strictness": 2,
                    "top_n_documents": 5,
                    "authentication": {
                    "type": "api_key",
                    "key": f"{search_key}"
                    }
                }
                }]
            }
        )
        
        response = f"{completion.choices[0].message.content}\n"

        # citations 출력 
        response2 = ""
        citations = completion.choices[0].message.context["citations"]
        for i in range(len(citations)):
            response2 += f"\n--- {i+1}번째 출처 ---\n{citations[i]['content']}\n"
        
        return response, response2

    except Exception as ex:
        return ex

In [156]:
response, response2 = request_SDK("런던에 대한 정보를 소개해줘")
print(response)
print(response2)

런던은 영국의 수도로, 풍부한 역사와 문화유산을 가진 도시입니다. 이 도시는 다양한 관광 명소와 활동으로 유명합니다. 예를 들어, 런던에는 세계적으로 유명한 박물관, 갤러리, 그리고 다양한 레스토랑과 상점들이 있습니다 [doc1].

런던의 기후는 대체로 온화하며, 여름철에는 기온이 20도에서 25도 사이로 올라가는 경우가 많고, 겨울철에는 기온이 0도에서 10도 사이로 내려갑니다 [doc1]. 또한, 런던은 다양한 문화 행사와 축제가 열리는 곳으로, 매년 많은 관광객들이 방문합니다 [doc1].

여행을 계획할 때는 런던의 교통 시스템을 이용하는 것이 편리하며, 지하철과 버스가 잘 발달되어 있습니다 [doc1]. 런던을 방문할 때는 반드시 유명한 관광지인 빅벤, 타워 브리지, 그리고 런던 아이를 체크해보는 것이 좋습니다 [doc1]. 

더 많은 정보는 Margie's Travel 웹사이트를 통해 확인할 수 있습니다 [doc1].


--- 1번째 출처 ---
Margie’s Travel Presents…

런던, London
런던은 영국과 영국의 
수도이자 가장 인구가 많은 
도시입니다. 영국 섬 
남동쪽 템즈강에 위치한 
런던은 2천 년 동안 주요 
정착지였습니다. 런던은 
로마인들에 의해 
설립되었으며, 로마인들은 
런던의 이름을 
'론디니움'으로 
명명했습니다.
런던의 고대 중심지인 
런던시는 대체로 
1.12제곱마일의 중세 
경계를 유지하고 있습니다.
적어도 19세기부터 런던은 역사적으로 미들섹스, 에식스, 서리, 켄트, 하트퍼드셔로 나뉜 이 중심부 
주변의 대도시를 지칭해 왔으며, 오늘날 런던 시장과 런던 의회가 통치하는 대런던을 대부분 차지하고 
있습니다.

주로 인기 있는 것:
여가, 야외, 역사, 예술 및 문화

방문하기 가장 좋은 시기:
6월-8월
평균 강수량: 1.9 in 
평균 기온: 56-67°F

런던 호텔
마지 트래블은 런던에서 다음과 같은 숙박 옵션을 제공합니다:

버킹엄 호텔
버킹엄 궁전, 리젠트 파크, 트라팔가 광장 등 주요 

In [23]:
completion.choices[0].message.content

'남해에서 추천할 만한 여행지는 다음과 같습니다:\n\n1. **남해 독일마을**: 독일식 건축물과 독일 문화를 체험할 수 있는 곳으로, 다양한 카페와 레스토랑이 있어 여유롭게 시간을 보낼 수 있습니다. 독일 맥주와 소시지를 맛보는 것도 추천합니다 .\n\n2. **상주은모래비치**: 깨끗한 해변과 맑은 바다로 유명한 곳입니다. 해수욕과 다양한 해양 스포츠를 즐길 수 있으며, 주변에 아름다운 자연경관이 펼쳐져 있습니다 [doc2].\n\n3. **남해 바래길**: 자연 속에서 트레킹을 즐길 수 있는 길로, 다양한 코스가 마련되어 있어 걷는 즐거움을 느낄 수 있습니다. 특히 해안선을 따라 펼쳐진 경치가 아름답습니다 .\n\n4. **남해 금산**: 금산은 남해의 대표적인 명산으로, 정상에서 바라보는 경치가 일품입니다. 하이킹을 즐기며 자연을 만끽할 수 있습니다 [doc4].\n\n5. **남해 마늘소**: 남해에서 유명한 특산물 중 하나로, 마늘을 활용한 다양한 요리를 맛볼 수 있습니다. 특히 마늘 소스를 곁들인 해산물 요리가 인기입니다 .\n\n이 외에도 남해에는 다양한 관광명소와 맛집이 많으니, 여행 계획을 세우실 때 참고하시기 바랍니다!'

In [86]:
completion.choices[0].message.context["citations"][0]["content"]

'경상남도 남해군 남면 남서대로1179번길 40-109 아난티 남해 스위트 타워 7동\nY\nhttps://ananti.kr/ko\n34.7772024\n127.8530819\n이터널 저니 남해\n055-860-0688\n봄, 여름, 가을, 겨울, 자가용/렌트, 버스, 친구, 키즈, 연인, 우정女, 효도, 나홀로\n관광지'

## 2. REST API

샘플 코드

In [ ]:
api_base="https://7ai015-openai.openai.azure.com/"
api_key="***"
deployment_id="7ai015-gpt-4o-mini"
search_endpoint="https://7ai015-aisearch.search.windows.net"
search_key="***"
search_index="travel-index"

curl -i -X POST $api_base/openai/deployments/$deployment_id/chat/completions?api-version=2024-02-15-preview \
-H "Content-Type: application/json" \
-H "api-key: $api_key" \
-d '{
"messages": [
{
"role": "system",
"content": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다."
},
{
"role": "user",
"content": "남해 여행지 추천해줘"
},
{
"role": "assistant",
"content": "남해에는 여러 매력적인 여행지가 있습니다. 다음은 추천하는 몇 가지 장소입니다:\n\n1. **남해 독일마을**: 독일의 전통 건축 양식을 살린 이 마을은 아름다운 풍경과 함께 독일 문화 체험을 할 수 있는 곳입니다. 다양한 음식과 기념품 가게도 있어 관광객에게 인기가 많습니다 .\n\n2. **남해 바다**: 남해의 바다는 맑고 푸른 물과 함께 다양한 해양 스포츠를 즐길 수 있는 곳입니다. 해수욕을 하거나 스노클링을 즐기기에 좋은 장소입니다 [doc5].\n\n3. **상주은모래비치**: 이곳은 고운 모래와 잔잔한 파도가 매력적인 해변으로, 가족 단위 여행객에게 적합합니다. 여름철에는 많은 사람들이 방문합니다 .\n\n4. **남해 금산**: 금산은 아름다운 경치와 함께 하이킹을 즐길 수 있는 명소입니다. 정상에서는 남해의 전경을 한눈에 볼 수 있습니다 [doc3].\n\n5. **남해 물건리**: 이곳은 조용하고 평화로운 분위기로, 해변 산책이나 낚시를 즐기기에 적합한 곳입니다. 자연을 만끽하며 휴식을 취할 수 있습니다 .\n\n이 외에도 남해에는 다양한 관광 명소가 있으니, 여행 계획에 맞춰 방문해 보세요!"
}
],
"past_messages": 10,
"temperature": 0.5,
"top_p": 0.95,
"frequency_penalty": 0,
"presence_penalty": 0,
"max_tokens": 800,
"stop": null,
"azureSearchEndpoint": "https://7ai015-aisearch.search.windows.net",
"azureSearchKey": "****",
"azureSearchIndexName": "travel-index",
"data_sources": [
{
"type": "azure_search",
"parameters": {
"endpoint": "'$search_endpoint'",
"index_name": "travel-index",
"semantic_configuration": "travel-semantic",
"query_type": "semantic",
"fields_mapping": {},
"in_scope": true,
"role_information": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.",
"filter": null,
"strictness": 2,
"top_n_documents": 5,
"authentication": {
"type": "api_key",
"key": "QkjHtmZVcwnXNZDgY8uH46TkT5N3OkXauMa1OH0le2AzSeDXapGA"
},
"key": "'$search_key'",
"indexName": "'$search_index'"
}
}
]
}'

In [151]:
import requests

def request_REST(prompt):
  api_base="https://7ai015-openai.openai.azure.com/"
  api_key="937jKABfHgomOxXzVz4q7cxqEFwjMqrZ1HxIr4nUs4SfYQqD2SQrJQQJ99BFACfhMk5XJ3w3AAABACOG8ETF"
  deployment_id="7ai015-gpt-4o-mini"
  search_endpoint="https://7ai015-aisearch.search.windows.net"
  search_key="QkjHtmZVcwnXNZDgY8uH46TkT5N3OkXauMa1OH0le2AzSeDXapGA"
  search_index="travel-index"

  endpoint = f"{api_base}openai/deployments/{deployment_id}/chat/completions?api-version=2024-03-01-preview"


  headers = {
      "Content-Type": "application/json",
      "api-key": api_key
    }

  payload = {
    "messages": [
        {
          "role": "system",
          "content": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다."
        },
        {
          "role": "user",
          "content": prompt
        },
    ],
    "temperature": 0.5,
    "top_p": 0.95,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "max_tokens": 800,
    "stop": None,
    "data_sources": [
      {
        "type": "azure_search",
        "parameters": {
          "endpoint": search_endpoint,
          "index_name": search_index,
          "semantic_configuration": "travel-semantic",
          "query_type": "semantic",
          "fields_mapping": {},
          "in_scope": True,
          "filter": None,
          "strictness": 2,
          "top_n_documents": 5,
          "authentication": {
            "type": "api_key",
            "key": search_key
          }
        }
      }
    ]
  }
  response = requests.post(endpoint, headers=headers, json=payload)

  if response.status_code == 200:
    request_result = response.json()['choices'][0]['message']['content']
  else:
    request_result = 'requests Failed'

  # 답변 생성 
  response1 = f"{request_result}"

  # citations 추출
  response2 = ""
  citations = response.json()['choices'][0]['message']['context']['citations']
  for i, citation in enumerate(citations):
            response2 += f"\n--- {i+1}번째 출처 ---\n"

            lines = citation["content"].splitlines()
            for j, line in enumerate(lines):
                response2 += f"{j:02d}. {line}\n"

  return response1, response2

In [152]:
response1, response2 = request_REST("남해 여행지 추천해줘")
print(response1)
print(response2)

남해에서 추천할 만한 여행지는 다음과 같습니다:

1. **남해 독일마을**: 독일의 전통 건축 양식을 갖춘 마을로, 독일의 문화와 음식을 경험할 수 있는 곳입니다. 다양한 카페와 레스토랑이 있어 여유롭게 시간을 보낼 수 있습니다 .

2. **남해 금산**: 금산은 아름다운 자연 경관과 함께 트레킹을 즐길 수 있는 장소입니다. 특히, 정상에서 바라보는 경치는 매우 인상적입니다 .

3. **상주은모래비치**: 맑은 바다와 깨끗한 백사장이 매력적인 해변으로, 여름철에 많은 관광객이 찾는 명소입니다. 물놀이와 해양 스포츠를 즐기기에 좋습니다 [doc4].

4. **남해 바래길**: 바다를 따라 조성된 산책로로, 아름다운 해안선을 따라 걷는 즐거움을 제공합니다. 경치가 아름다워 사진 찍기에도 좋은 장소입니다 .

이 외에도 남해에는 다양한 관광지와 자연경관이 있으니, 여행 계획에 참고해 보세요!

--- 1번째 출처 ---
00. 경상남도 남해군 남면 남서대로1179번길 40-109 아난티 남해 스위트 타워 7동
01. Y
02. https://ananti.kr/ko
03. 34.7772024
04. 127.8530819
05. 이터널 저니 남해
06. 055-860-0688
07. 봄, 여름, 가을, 겨울, 자가용/렌트, 버스, 친구, 키즈, 연인, 우정女, 효도, 나홀로
08. 관광지

--- 2번째 출처 ---
00. 경상남도 남해군 이동면 용문사길 166-11
01. 산에 미친 사람들이 남해의 산을 등반한다면 분명 금산부터 탐낼 것이다. 그 다음으로 남해 최고봉인 망운산에 눈을 돌린 후, 돌아가는 시간에 쫓겨 황급히 섬을 떠난다. 이 때문에 남해의 잘록한 허리춤에 자리한 호구산(622m)은 그냥 스쳐 지나치기 쉬운 곳이다. 하지만 남해를 아는 사람들은 누구나 한번쯤 호구산의 거친 산세에 시선을 빼앗기게 된다. 지형도에는 산이름이 표기되어 있지 않지만, 송등산, 괴음산 등 산군을 엮어 호구산으로 대표되는 "호구산군립공원"으로 지정되어 있다. 남해군의 군립공

In [139]:
response.json()['choices'][0]['message']['context']['citations'][0]['content']

AttributeError: 'str' object has no attribute 'json'

In [83]:
len(response.json()['choices'][0]['message']['context']['citations'])

5

In [140]:
import requests

api_base="https://7ai015-openai.openai.azure.com/"
api_key="937jKABfHgomOxXzVz4q7cxqEFwjMqrZ1HxIr4nUs4SfYQqD2SQrJQQJ99BFACfhMk5XJ3w3AAABACOG8ETF"
deployment_id="7ai015-gpt-4o-mini"
search_endpoint="https://7ai015-aisearch.search.windows.net"
search_key="QkjHtmZVcwnXNZDgY8uH46TkT5N3OkXauMa1OH0le2AzSeDXapGA"
search_index="travel-index"

endpoint = f"{api_base}openai/deployments/{deployment_id}/chat/completions?api-version=2024-03-01-preview"


headers = {
    "Content-Type": "application/json",
    "api-key": api_key
  }

payload = {
  "messages": [
      {
        "role": "system",
        "content": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다."
      },
      {
        "role": "user",
        "content": "남해 여행지 추천해줘"
      },
  ],
  "temperature": 0.5,
  "top_p": 0.95,
  "frequency_penalty": 0,
  "presence_penalty": 0,
  "max_tokens": 800,
  "stop": None,
  "data_sources": [
    {
      "type": "azure_search",
      "parameters": {
        "endpoint": search_endpoint,
        "index_name": search_index,
        "semantic_configuration": "travel-semantic",
        "query_type": "semantic",
        "fields_mapping": {},
        "in_scope": True,
        "filter": None,
        "strictness": 2,
        "top_n_documents": 5,
        "authentication": {
          "type": "api_key",
          "key": search_key
        }
      }
    }
  ]
}
response = requests.post(endpoint, headers=headers, json=payload)

if response.status_code == 200:
  request_result = response.json()['choices'][0]['message']['content']
else:
  request_result = 'requests Failed'

# 답변 생성 
print(f"{request_result}\n")

# citations 추출
citations = response.json()['choices'][0]['message']['context']['citations']
for i in range(len(citations)):
  print(f"{i}번째 citation: {citations[i]['content']}\n")

남해에는 다양한 여행지가 있습니다. 아래는 추천할 만한 몇 가지 장소입니다:

1. **남해 다랭이마을**: 이곳은 계단식 논과 아름다운 경관으로 유명합니다. 특히 일출과 일몰이 아름다워 많은 관광객들이 찾습니다 .

2. **남해 독일마을**: 독일식 건축물이 있는 이 마을은 독일의 문화와 역사를 느낄 수 있는 곳입니다. 다양한 음식과 기념품 가게가 있어 즐길 거리가 많습니다 [doc2].

3. **상주 은모래비치**: 맑고 깨끗한 바다와 고운 모래사장이 있는 해변으로, 휴식을 취하기에 좋은 장소입니다. 여름철에는 수영과 해양 스포츠를 즐길 수 있습니다 [doc3].

4. **남해 금산**: 금산은 등산로와 아름다운 경치를 제공하는 곳으로, 정상에서 바라보는 바다의 경관이 특히 인상적입니다 [doc4].

5. **남해 보리암**: 이곳은 바다를 내려다보는 절경과 함께 조용한 분위기를 제공합니다. 특히 사찰의 아름다움과 자연이 어우러져 힐링하기 좋은 장소입니다 .

각 장소마다 독특한 매력이 있으니, 여행 계획에 맞춰 선택하시면 좋겠습니다!

0번째 citation: 경상남도 남해군 남면 남서대로1179번길 40-109 아난티 남해 스위트 타워 7동
Y
https://ananti.kr/ko
34.7772024
127.8530819
이터널 저니 남해
055-860-0688
봄, 여름, 가을, 겨울, 자가용/렌트, 버스, 친구, 키즈, 연인, 우정女, 효도, 나홀로
관광지

1번째 citation: 경상남도 남해군 이동면 용문사길 166-11
산에 미친 사람들이 남해의 산을 등반한다면 분명 금산부터 탐낼 것이다. 그 다음으로 남해 최고봉인 망운산에 눈을 돌린 후, 돌아가는 시간에 쫓겨 황급히 섬을 떠난다. 이 때문에 남해의 잘록한 허리춤에 자리한 호구산(622m)은 그냥 스쳐 지나치기 쉬운 곳이다. 하지만 남해를 아는 사람들은 누구나 한번쯤 호구산의 거친 산세에 시선을 빼앗기게 된다. 지형도에는 산이름이 표기되어 있지 않지만, 송등산, 괴음산 등 산군을 엮어

In [148]:
response.json()['choices'][0]['message']['context']

{'citations': [{'content': '경상남도 남해군 남면 남서대로1179번길 40-109 아난티 남해 스위트 타워 7동\nY\nhttps://ananti.kr/ko\n34.7772024\n127.8530819\n이터널 저니 남해\n055-860-0688\n봄, 여름, 가을, 겨울, 자가용/렌트, 버스, 친구, 키즈, 연인, 우정女, 효도, 나홀로\n관광지',
   'title': None,
   'url': None,
   'filepath': None,
   'chunk_id': '0'},
  {'content': '경상남도 남해군 이동면 용문사길 166-11\n산에 미친 사람들이 남해의 산을 등반한다면 분명 금산부터 탐낼 것이다. 그 다음으로 남해 최고봉인 망운산에 눈을 돌린 후, 돌아가는 시간에 쫓겨 황급히 섬을 떠난다. 이 때문에 남해의 잘록한 허리춤에 자리한 호구산(622m)은 그냥 스쳐 지나치기 쉬운 곳이다. 하지만 남해를 아는 사람들은 누구나 한번쯤 호구산의 거친 산세에 시선을 빼앗기게 된다. 지형도에는 산이름이 표기되어 있지 않지만, 송등산, 괴음산 등 산군을 엮어 호구산으로 대표되는 "호구산군립공원"으로 지정되어 있다. 남해군의 군립공원으로 지정된 것은 그만큼 자연경관이 뛰어나고 보존가치가 있는 곳이라는 말이다. 호구산이란 이름은 산이 호랑이 형상을 닮아서 붙여졌다는 설과 옛날 지리산에서 건너온 호랑이가 이 산에 살아서 호구산이라 했다는 설이 전해진다.\nY\n34.7879404\n127.9230404\n호구산\n055-860-3676\n봄, 여름, 가을, 자가용/렌트, 친구, 우정男, 나홀로\n관광지',
   'title': None,
   'url': None,
   'filepath': None,
   'chunk_id': '0'},
  {'content': '남해군 삼동면 동부대로1030번길 59\n2천여 그루의 방풍림으로 조성된 울창한 수림이 초승달 모양을 그리며 장관을 이루고 있는 곳으로 숲과 어우러진 몽돌해안은 남해 

In [ ]:
citations = completion.choices[0].message.context['citations']
for citation in citations:
    print(citation['content'] + '\n')